In [243]:
from subprocess import CREATE_NO_WINDOW
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import *

from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup as bs
from datetime import datetime

import pandas as pd
import time
import pyperclip
import os

In [2]:
def find_css(css_selector, browser):
    return browser.find_element(By.CSS_SELECTOR, css_selector)

def finds_css(css_selector, browser):
    return browser.find_elements(By.CSS_SELECTOR, css_selector)

def find_xpath(xpath, browser):
    return browser.find_element(By.XPATH, xpath)

def finds_xpath(xpath, browser):
    return browser.find_elements(By.XPATH, xpath)

def find_id(e_id, browser):
    return browser.find_element(By.ID, e_id)

def find_className(cn, browser):
    return browser.find_element(By.CLASS_NAME, cn)

def finds_className(cn , browser):
    return browser.find_elements(By.CLASS_NAME, cn)

def find_linktext(lt, browser):
    return browser.find_element(By.LINK_TEXT, lt)

In [402]:
def open_browser():
    options = webdriver.ChromeOptions()
    options.add_argument('--no--sandbox')
    options.add_argument('no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--window-size=1080,800')
    options.add_argument('incognito')

    chrome_service = Service('chromedriver')
    chrome_service.creationflags = CREATE_NO_WINDOW
    chrome_service = Service(executable_path="chromedriver.exe")
    browser = webdriver.Chrome(service=chrome_service, options=options)
    
    browser.get('https://www.instagram.com/')
    browser.implicitly_wait(2)
    
    return browser

In [ ]:
INPUT_ID = '//*[@id="loginForm"]/div/div[1]/div/label/input'
INPUT_PWD = '//*[@id="loginForm"]/div/div[2]/div/label/input'
ALERT_2 = '/html/body/div[2]/div/div/div[3]/div/div/div[1]/div/div[2]/div/div/div/div/div[2]/div/div/div[3]/button[2]'

FIRST_POST = '/html/body/div[2]/div/div/div[2]/div/div/div/div[1]/div[1]/div[2]/section/main/article/div/div/div/div[1]/div[1]'
WAIT_UNTIL_LOADING = '/html/body/div[2]/div/div/div[2]/div/div/div/div[1]/div[1]/div[2]/div[2]/section/main/div/header/div/div/span/img'
NEXT_FOR_FIRST = '/html/body/div[2]/div/div/div[3]/div/div/div[1]/div/div[3]/div/div/div/div/div[1]/div/div/div'
NEXT_AFTER_FIRST = '/html/body/div[2]/div/div/div[3]/div/div/div[1]/div/div[3]/div/div/div/div/div[1]/div/div/div[2]'

In [471]:
insta_id = 'mirae_market_09'
insta_pwd = 'han8289988@'


In [472]:
keyword = '맛집'

In [478]:
user_id = []
COUNT = 5
FOLLOWER = 200

In [474]:
browser = open_browser()

In [475]:
def login(browser, insta_id, insta_pwd):
    input_id = find_xpath(INPUT_ID, browser)
    input_pw = find_xpath(INPUT_PWD, browser)

    time.sleep(2)

    pyperclip.copy(insta_id)
    input_id.send_keys(Keys.CONTROL, "v")

    pyperclip.copy(insta_pwd) 
    input_pw.send_keys(Keys.CONTROL, "v")
    input_pw.send_keys("\n")

    try:
        if find_id('slfErrorAlert', browser).text == '잘못된 비밀번호입니다. 다시 확인하세요.':
            print('로그인 실패')
    except:
        pass

    alert_1 = find_className('_ac8f', browser).click()

    try:
        alert_2 = find_xpath(ALERT_2, browser).click()
    except:
        pass

In [480]:

def get_user_ids(browser, keyword, COUNT, FOLLOWER):
    browser.get(f'https://www.instagram.com/explore/tags/{keyword}/')

    time.sleep(5)

    first_post = find_xpath(FIRST_POST, browser).click()

    soup = bs(browser.page_source, "html.parser")
    s = soup.find_all(class_="xt0psk2")

    for i in s:
        try:
            if i.find("a").text:
                tmp = i.find('a'). text
                break
        except:
            pass
    print(tmp)

    browser.execute_script('window.open("https://google.com");')  #구글 창 새 탭으로 열기
    time.sleep(1)

    browser.switch_to.window(browser.window_handles[-1])  #새로 연 탭으로 이동
    browser.get(f'https://www.instagram.com/{tmp}/')
    browser.implicitly_wait(10)
    find_xpath(WAIT_UNTIL_LOADING,browser)

    soup = bs(browser.page_source, "html.parser")
    s = soup.find_all(class_="_ac2a")

    for i in s:
        print(i.text)

    f = s[1].text
    if '만' in f:
        if '.' in f:
            f = f.replace('.','').replace('만', '000')
        else:
            f = f.replace('만', '0000')

    if int(f) >= FOLLOWER:
        print('add')
        user_id.append(tmp)
        user_id = list(set(user_id))

    browser.close()  #링크 이동 후 탭 닫기
    browser.switch_to.window(browser.window_handles[-1])  #다시 이전 창(탭)으로 이동
    time.sleep(1)

    if len(user_id) >= COUNT:
        print('out')
    else:
        next_for_first = find_xpath(NEXT_FOR_FIRST, browser).click()

        while True:
            soup = bs(browser.page_source, "html.parser")
            s = soup.find_all(class_="xt0psk2")

            for i in s:
                try:
                    if i.find("a").text:
                        tmp = i.find('a'). text
                        break
                except:
                    pass
            print(tmp)
            if tmp in user_id:
                continue

            browser.execute_script('window.open("https://google.com");')  #구글 창 새 탭으로 열기
            time.sleep(1)

            browser.switch_to.window(browser.window_handles[-1])  #새로 연 탭으로 이동

            browser.get(f'https://www.instagram.com/{tmp}/')
            browser.implicitly_wait(10)
            find_xpath(WAIT_UNTIL_LOADING,browser)
            soup = bs(browser.page_source, "html.parser")
            s = soup.find_all(class_="_ac2a")

            for i in s:
                print(i.text)

            f = s[1].text
            if '만' in f:
                if '.' in f:
                    f = f.replace('.','').replace('만', '000')
                else:
                    f = f.replace('만', '0000')

            if int(f) >= FOLLOWER:
                print('add')
                user_id.append(tmp)
                user_id = list(set(user_id))
                if len(user_id) >= COUNT:
                        print('out')
                        browser.close()  #링크 이동 후 탭 닫기
                        browser.switch_to.window(browser.window_handles[-1]) 
                        break

            else: print('pass')
            browser.close()  #링크 이동 후 탭 닫기
            browser.switch_to.window(browser.window_handles[-1])  #다시 이전 창(탭)으로 이동
            time.sleep(1)

            try:
                next_after_first = find_xpath(NEXT_AFTER_FIRST, browser).click()
            except NoSuchElementException:
                break
                
    return user_id

7.april.park
32
1134
294
add
min_happy.matzip
541
3.4만
4952
add
jinwon5385
952
4.2만
1874
add
mat__.zip
387
4.2만
542
add
wetonwet2
1056
9725
6286
add
out
min_happy.matzip
wetonwet2
7.april.park
jinwon5385
mat__.zip


In [470]:

        browser.close()  #링크 이동 후 탭 닫기
        browser.switch_to.window(browser.window_handles[-1])

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=114.0.5735.199)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00E1A813+48355]
	(No symbol) [0x00DAC4B1]
	(No symbol) [0x00CB5358]
	(No symbol) [0x00C9D293]
	(No symbol) [0x00CFE37B]
	(No symbol) [0x00D02842]
	(No symbol) [0x00CFA703]
	(No symbol) [0x00CD82DC]
	(No symbol) [0x00CD93DD]
	GetHandleVerifier [0x0107AABD+2539405]
	GetHandleVerifier [0x010BA78F+2800735]
	GetHandleVerifier [0x010B456C+2775612]
	GetHandleVerifier [0x00EA51E0+616112]
	(No symbol) [0x00DB5F8C]
	(No symbol) [0x00DB2328]
	(No symbol) [0x00DB240B]
	(No symbol) [0x00DA4FF7]
	BaseThreadInitThunk [0x759300C9+25]
	RtlGetAppContainerNamedObjectPath [0x76F67B1E+286]
	RtlGetAppContainerNamedObjectPath [0x76F67AEE+238]


In [446]:
soup = bs(browser.page_source, "html.parser")
s = soup.find_all(class_="_ac2a")

In [447]:
for i in s:
    print(i.text)

527
1.4만
6106


In [448]:
a = s[1].text

In [449]:
if '.' in a:
    a = a.replace('.','').replace('만', '000')

In [450]:
a

'14000'

In [463]:
f.replace('만', '0000')

'10000'

In [466]:
if '만' in f:
    print(1)
    if '.' in f:
        f = f.replace('.','').replace('만', '000')
        print(2)
    else:
        f = f.replace('만', '0000')
        print(3)

1
3


In [464]:
f = '1만'

In [467]:
f

'1만'